# Population vs House Price Analysis

This notebook analyzes the relationship between municipal population and house prices in Belgium.

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

## 1. Load Data

In [10]:
# Load population data
pop_df = pd.read_csv('../data/raw/TF_SOC_POP_STRUCT_2025.csv')

# Load property data
prop_df = pd.read_csv('../data/raw/combined_properties.csv', low_memory=False)

print(f"Population data shape: {pop_df.shape}")
print(f"Property data shape: {prop_df.shape}")

Population data shape: (466822, 21)
Property data shape: (10947, 104)


## 2. Process Population Data

Aggregate population by municipality by summing all MS_POPULATION values.

In [11]:
# Group by municipality and sum population
# Use TX_DESCR_NL (Dutch names) as primary identifier
pop_by_municipality = pop_df.groupby('TX_DESCR_NL').agg({
    'MS_POPULATION': 'sum',
    'TX_DESCR_FR': 'first',  # Keep French name for reference
    'CD_REFNIS': 'first'  # Keep municipality code
}).reset_index()

pop_by_municipality.columns = ['municipality_nl', 'total_population', 'municipality_fr', 'refnis_code']

print(f"Number of municipalities: {len(pop_by_municipality)}")
print(f"\nTop 10 municipalities by population:")
print(pop_by_municipality.nlargest(10, 'total_population')[['municipality_nl', 'total_population']])

Number of municipalities: 565

Top 10 municipalities by population:
    municipality_nl  total_population
19        Antwerpen            562002
182            Gent            272657
92        Charleroi            205763
83          Brussel            198314
323            Luik            197323
443      Schaarbeek            129775
13       Anderlecht            128724
81           Brugge            120283
362           Namen            115029
299          Leuven            104906


## 3. Process Property Data

Calculate median and mean house prices by municipality.

In [12]:
# Remove properties with missing or zero prices
prop_df_clean = prop_df[prop_df['price'].notna() & (prop_df['price'] > 0)].copy()

# Normalize municipality names (lowercase for matching)
prop_df_clean['municipality_lower'] = prop_df_clean['municipality'].str.lower().str.strip()

# Group by municipality and calculate price statistics
price_by_municipality = prop_df_clean.groupby('municipality_lower').agg({
    'price': ['median', 'mean', 'count', 'std']
}).reset_index()

price_by_municipality.columns = ['municipality_lower', 'median_price', 'mean_price', 'property_count', 'price_std']

print(f"Number of municipalities with properties: {len(price_by_municipality)}")
print(f"Total properties analyzed: {price_by_municipality['property_count'].sum()}")
print(f"\nTop 10 municipalities by median price:")
print(price_by_municipality.nlargest(10, 'median_price')[['municipality_lower', 'median_price', 'property_count']])

Number of municipalities with properties: 272
Total properties analyzed: 431

Top 10 municipalities by median price:
    municipality_lower  median_price  property_count
82              elsene     2350000.0               4
159          linkebeek     2290000.0               1
244            tremelo     2195000.0               1
75             donceel     1999999.0               1
37            borgloon     1999000.0               1
54              celles     1750000.0               1
110          harelbeke     1650000.0               1
48            brussels     1495000.0               1
247              ukkel     1485000.0              12
34                biez     1250000.0               1


## 4. Merge Population and Price Data

In [13]:
# Normalize population municipality names for matching
pop_by_municipality['municipality_lower'] = pop_by_municipality['municipality_nl'].str.lower().str.strip()

# Also try French names for better matching
pop_by_municipality['municipality_fr_lower'] = pop_by_municipality['municipality_fr'].str.lower().str.strip()

# Merge datasets
merged_df = price_by_municipality.merge(
    pop_by_municipality[['municipality_lower', 'total_population', 'municipality_nl']],
    on='municipality_lower',
    how='inner'
)

# Try to match unmatched municipalities using French names
unmatched = price_by_municipality[~price_by_municipality['municipality_lower'].isin(merged_df['municipality_lower'])]
additional_matches = unmatched.merge(
    pop_by_municipality[['municipality_fr_lower', 'total_population', 'municipality_nl']],
    left_on='municipality_lower',
    right_on='municipality_fr_lower',
    how='inner'
).drop('municipality_fr_lower', axis=1)

# Combine matches
merged_df = pd.concat([merged_df, additional_matches], ignore_index=True)

print(f"Successfully matched municipalities: {len(merged_df)}")
print(f"Total properties in matched municipalities: {merged_df['property_count'].sum()}")
print(f"\nSample of merged data:")
print(merged_df.head(10))

Successfully matched municipalities: 162
Total properties in matched municipalities: 297

Sample of merged data:
  municipality_lower  median_price  mean_price  property_count      price_std  \
0             aalter      535000.0    535000.0               2  155563.491861   
1           affligem      295000.0    295000.0               1            NaN   
2              alken      650000.0    650000.0               1            NaN   
3               amay      157000.0    157000.0               2  110308.657865   
4            andenne      317500.0    317500.0               2   31819.805153   
5         anderlecht      520000.0    581500.0               3  171243.248042   
6            anzegem      690000.0    690000.0               1            NaN   
7               asse      950000.0    848750.0               4  202500.000000   
8           assenede      375000.0    375000.0               1            NaN   
9            avelgem      458000.0    458000.0               1            NaN

## 5. Statistical Analysis

In [14]:
# Calculate correlation
correlation_median = merged_df['total_population'].corr(merged_df['median_price'])
correlation_mean = merged_df['total_population'].corr(merged_df['mean_price'])

# Perform linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(
    merged_df['total_population'], 
    merged_df['median_price']
)

print("=" * 50)
print("CORRELATION ANALYSIS")
print("=" * 50)
print(f"Correlation (Population vs Median Price): {correlation_median:.4f}")
print(f"Correlation (Population vs Mean Price): {correlation_mean:.4f}")
print(f"\nLinear Regression (Population vs Median Price):")
print(f"  R-squared: {r_value**2:.4f}")
print(f"  P-value: {p_value:.6f}")
print(f"  Slope: {slope:.4f}")
print(f"  Interpretation: {'Statistically significant' if p_value < 0.05 else 'Not statistically significant'}")

# Summary statistics
print("\n" + "=" * 50)
print("SUMMARY STATISTICS")
print("=" * 50)
print(merged_df[['total_population', 'median_price', 'mean_price', 'property_count']].describe())

CORRELATION ANALYSIS
Correlation (Population vs Median Price): 0.2143
Correlation (Population vs Mean Price): 0.2406

Linear Regression (Population vs Median Price):
  R-squared: 0.0459
  P-value: 0.006170
  Slope: 4.0262
  Interpretation: Statistically significant

SUMMARY STATISTICS
       total_population  median_price    mean_price  property_count
count        162.000000  1.620000e+02  1.620000e+02      162.000000
mean       23300.864198  5.123559e+05  5.197067e+05        1.833333
std        20137.090416  3.783007e+05  3.738802e+05        1.508774
min         2512.000000  6.000000e+04  6.000000e+04        1.000000
25%        11600.000000  2.978750e+05  3.118750e+05        1.000000
50%        18148.000000  4.325000e+05  4.325000e+05        1.000000
75%        28692.000000  5.761250e+05  5.841250e+05        2.000000
max       129775.000000  2.350000e+06  2.290000e+06       12.000000


## 6. Visualizations

In [15]:
# Simple scatter plot
fig1 = px.scatter(
    merged_df,
    x='total_population',
    y='median_price',
    title='Population vs Median House Price',
    labels={
        'total_population': 'Population',
        'median_price': 'Median Price (€)'
    }
)

fig1.show()

# Simple scatter plot with log scale
fig2 = px.scatter(
    merged_df,
    x='total_population',
    y='median_price',
    title='Population vs Median House Price (Log Scale)',
    labels={
        'total_population': 'Population (log)',
        'median_price': 'Median Price (€)'
    },
    log_x=True
)

fig2.show()

In [16]:
# Population categories
merged_df['pop_category'] = pd.cut(
    merged_df['total_population'], 
    bins=[0, 10000, 50000, 100000, float('inf')],
    labels=['<10K', '10K-50K', '50K-100K', '>100K']
)

# Simple histogram of population
fig1 = px.histogram(
    merged_df,
    x='total_population',
    title='Distribution of Municipal Population',
    labels={'total_population': 'Population'}
)
fig1.show()

# Simple histogram of prices
fig2 = px.histogram(
    merged_df,
    x='median_price',
    title='Distribution of Median House Prices',
    labels={'median_price': 'Median Price (€)'}
)
fig2.show()

# Simple bar chart by category
pop_category_prices = merged_df.groupby('pop_category', observed=True)['median_price'].median()
fig3 = px.bar(
    x=pop_category_prices.index.astype(str),
    y=pop_category_prices.values,
    title='Median Price by Population Category',
    labels={'x': 'Population Category', 'y': 'Median Price (€)'}
)
fig3.show()

# Simple box plot
fig4 = px.box(
    merged_df,
    x='pop_category',
    y='median_price',
    title='Price Distribution by Population Category',
    labels={'pop_category': 'Population Category', 'median_price': 'Median Price (€)'}
)
fig4.show()

## 7. Top and Bottom Municipalities

In [17]:
# Top 10 most expensive municipalities
print("=" * 80)
print("TOP 10 MOST EXPENSIVE MUNICIPALITIES")
print("=" * 80)
top_10_expensive = merged_df.nlargest(10, 'median_price')[['municipality_nl', 'total_population', 'median_price', 'property_count']]
print(top_10_expensive.to_string(index=False))

print("\n" + "=" * 80)
print("TOP 10 LEAST EXPENSIVE MUNICIPALITIES")
print("=" * 80)
bottom_10_expensive = merged_df.nsmallest(10, 'median_price')[['municipality_nl', 'total_population', 'median_price', 'property_count']]
print(bottom_10_expensive.to_string(index=False))

print("\n" + "=" * 80)
print("TOP 10 MOST POPULOUS MUNICIPALITIES")
print("=" * 80)
top_10_populous = merged_df.nlargest(10, 'total_population')[['municipality_nl', 'total_population', 'median_price', 'property_count']]
print(top_10_populous.to_string(index=False))

TOP 10 MOST EXPENSIVE MUNICIPALITIES
municipality_nl  total_population  median_price  property_count
         Elsene             89897     2350000.0               4
      Linkebeek              4646     2290000.0               1
        Tremelo             15418     2195000.0               1
        Donceel              3128     1999999.0               1
         Celles              5775     1750000.0               1
      Harelbeke             30311     1650000.0               1
          Ukkel             87194     1485000.0              12
     Schaarbeek            129775     1224500.0               2
Wezembeek-Oppem             14760     1095000.0               1
         Rebecq             11185     1090000.0               1

TOP 10 LEAST EXPENSIVE MUNICIPALITIES
municipality_nl  total_population  median_price  property_count
     Courcelles             31436       60000.0               1
          Awans              9373      140000.0               1
      Quaregnon             

## 8. Conclusions

This analysis examines the relationship between municipal population and house prices:

1. **Correlation**: The correlation coefficient indicates the strength and direction of the linear relationship
2. **Statistical Significance**: The p-value tells us if the relationship is statistically significant
3. **Price Distribution**: Shows how house prices vary across municipalities of different population sizes
4. **Regional Patterns**: Identifies which municipalities have the highest and lowest prices relative to their population